<a href="https://colab.research.google.com/github/TravellerOnCode/Credit-Card-Default-Risk---AmEx/blob/main/Credit_Card_Default_Risk_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement

The problem statement can be accessed from [here](https://www.hackerearth.com/challenges/competitive/amexpert-code-lab/machine-learning/credit-card-default-risk-5-95cbc85f/).


**References**


*   https://www.analyticsvidhya.com/blog/2021/10/handling-missing-value/
*   https://arxiv.org/pdf/1711.00837.pdf
*   https://youtu.be/oJvjRnuoqQM



# Imports

In [ ]:
pip install kmeans_smote;

In [ ]:
import requests, zipfile, io
import pandas as pd
pd.options.mode.chained_assignment = None

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


# Load the Dataset

In [ ]:
#the csv is stored in a github repo
r_train = requests.get('https://github.com/TravellerOnCode/Credit-Card-Default-Risk---AmEx/blob/main/Data/train.csv?raw=true')
r_test = requests.get('https://github.com/TravellerOnCode/Credit-Card-Default-Risk---AmEx/blob/main/Data/test.csv?raw=true') 

df_train = pd.read_csv(io.BytesIO(r_train.content))
df_test = pd.read_csv(io.BytesIO(r_test.content))

print('\nTraining Dataset : ',df_train.shape)
df_train.head(3)


Training Dataset :  (45528, 19)


,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,CST_115179,ita Bose,46,F,N,Y,0.0,107934.04,612.0,Unknown,1.0,1.0,33070.28,18690.93,73,544.0,2,1,1
1,CST_121920,Alper Jonathan,29,M,N,Y,0.0,109862.62,2771.0,Laborers,2.0,0.0,15329.53,37745.19,52,857.0,0,0,0
2,CST_109330,Umesh Desai,37,M,N,Y,0.0,230153.17,204.0,Laborers,2.0,0.0,48416.60,41598.36,43,650.0,0,0,0


In [ ]:
print('\nTest Dataset : ',df_test.shape)
df_test.head(3)


Test Dataset :  (11383, 18)


,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months
0,CST_142525,Siva,52,F,Y,N,0.0,232640.53,998.0,Unknown,2.0,0.0,14406.73,26524.40,4,779.0,0,0
1,CST_129215,Scott,48,F,N,N,1.0,284396.79,1338.0,Unknown,3.0,0.0,57479.99,68998.72,70,806.0,0,0
2,CST_138443,Victoria,50,F,N,N,1.0,149419.28,1210.0,Unknown,3.0,0.0,21611.01,25187.80,71,528.0,2,0


In [ ]:
df_train['credit_card_default'].value_counts()

0    41831
1     3697
Name: credit_card_default, dtype: int64

In [ ]:
df_train_backup = df_train
df_test_backup = df_test

In [ ]:
#Percentage of null values present in each column --- TRAIN DATASET
((df_test.isnull() | df_test.isna()).sum() * 100 / df_test.index.size).round(2)

customer_id                0.00
name                       0.00
age                        0.00
gender                     0.00
owns_car                   1.16
owns_house                 0.00
no_of_children             1.67
net_yearly_income          0.00
no_of_days_employed        0.92
occupation_type            0.00
total_family_members       0.27
migrant_worker             0.23
yearly_debt_payments       0.19
credit_limit               0.00
credit_limit_used(%)       0.00
credit_score               0.03
prev_defaults              0.00
default_in_last_6months    0.00
dtype: float64

In [ ]:
#Percentage of null values present in each column --- TRAIN DATASET
((df_train.isnull() | df_train.isna()).sum() * 100 / df_train.index.size).round(2)

customer_id                0.00
name                       0.00
age                        0.00
gender                     0.00
owns_car                   1.20
owns_house                 0.00
no_of_children             1.70
net_yearly_income          0.00
no_of_days_employed        1.02
occupation_type            0.00
total_family_members       0.18
migrant_worker             0.19
yearly_debt_payments       0.21
credit_limit               0.00
credit_limit_used(%)       0.00
credit_score               0.02
prev_defaults              0.00
default_in_last_6months    0.00
credit_card_default        0.00
dtype: float64

In [ ]:
#separate the dataset
default_df = df_train.loc[df_train['credit_card_default'] == 1]
non_default_df = df_train.loc[df_train['credit_card_default'] == 0]

In [ ]:
#Percentage of null values present in each column --- TRAIN DATASET
((non_default_df.isnull() | non_default_df.isna()).sum() * 100 / non_default_df.index.size).round(2)

customer_id                0.00
name                       0.00
age                        0.00
gender                     0.00
owns_car                   1.20
owns_house                 0.00
no_of_children             1.69
net_yearly_income          0.00
no_of_days_employed        1.02
occupation_type            0.00
total_family_members       0.18
migrant_worker             0.19
yearly_debt_payments       0.21
credit_limit               0.00
credit_limit_used(%)       0.00
credit_score               0.02
prev_defaults              0.00
default_in_last_6months    0.00
credit_card_default        0.00
dtype: float64

In [ ]:
#Percentage of null values present in each column --- TRAIN DATASET
((default_df.isnull() | default_df.isna()).sum() * 100 / default_df.index.size).round(2)

customer_id                0.00
name                       0.00
age                        0.00
gender                     0.00
owns_car                   1.24
owns_house                 0.00
no_of_children             1.79
net_yearly_income          0.00
no_of_days_employed        0.95
occupation_type            0.00
total_family_members       0.19
migrant_worker             0.16
yearly_debt_payments       0.22
credit_limit               0.00
credit_limit_used(%)       0.00
credit_score               0.03
prev_defaults              0.00
default_in_last_6months    0.00
credit_card_default        0.00
dtype: float64

In [ ]:
non_na_df = df_train.dropna()
non_na_df['credit_card_default'].value_counts()

0    39978
1     3531
Name: credit_card_default, dtype: int64

**Global Declarations**

In [ ]:
NON_FEATURE_COLS = ['customer_id', 'name', 'occupation_type']
LABEL_COL = 'credit_card_default'
CATEGORICAL_COLS = ['owns_car','owns_house','gender']

*The % of NULL values is less than 1.5% in every column. Hence we need not drop a column.*

# Handle missing values

In [ ]:
def replace_NA(df):

  #Replace the missing values for categorical columns with mode
  df['owns_car'] = df['owns_car'].fillna(df['owns_car'].mode()[0])

  #Replace the missing values with mean
  df['no_of_days_employed'] = df['no_of_days_employed'].fillna(df['no_of_days_employed'].mean())
  df['no_of_children'] = df['no_of_children'].fillna(df['no_of_children'].mean())
  df['total_family_members'] = df['total_family_members'].fillna(df['total_family_members'].mean())
  df['migrant_worker'] = df['migrant_worker'].fillna(df['migrant_worker'].mean())
  df['yearly_debt_payments'] = df['yearly_debt_payments'].fillna(df['yearly_debt_payments'].mean())
  df['credit_score'] = df['credit_score'].fillna(df['credit_score'].mean())

  return df

# Data Transformation

In [ ]:
#Encode Columns
def encode_columns(df_train,df_test):
  # instantiate labelencoder object
  le = LabelEncoder()

  # apply le on categorical feature columns

  X_train = df_train 
  X_train[CATEGORICAL_COLS] = X_train[CATEGORICAL_COLS].apply(lambda col: le.fit_transform(col))

  X_test = df_test
  X_test[CATEGORICAL_COLS] = X_test[CATEGORICAL_COLS].apply(lambda col: le.fit_transform(col))

  return (X_train,X_test)

In [ ]:
def create_features_and_labels(df,train):
  if train == 'X':
    df = df.sample(frac=1).reset_index(drop=True)
    x_features = df.drop(NON_FEATURE_COLS,axis=1)
    x_features = x_features.drop([LABEL_COL],axis=1)
    y_label = df[LABEL_COL]
    print(x_features.shape , y_label.shape)
    return (x_features,y_label)
  else:
    x_features = df.drop(NON_FEATURE_COLS,axis=1)
    print(x_features.shape)
    return (x_features)

# Prediction & Evaluation Metrics

In [ ]:
def generate_predictions(model,x_test,y_test):

  predictions = model.predict(x_test)
  score = model.score(x_test,y_test) * 100
  print('Accuracy = ',score)

  print('Classification Report')
  print(classification_report(y_test, predictions))

# Baseline Model (Logistic Regression)

In [ ]:
def baseline_model(x_train,y_train):
  #Load the Model
  lr = LogisticRegression()
  #Train
  lr.fit(x_train,y_train)

  return lr

# Random Forest

In [ ]:
#hyperparameters
params_rf = {}

In [ ]:
#Random Forest
def random_forest(x_train,y_train,estimators):
  rfc = RandomForestClassifier(n_estimators = estimators)
  rfc.fit(x_train,y_train)

  return rfc

# Prediction on given Data

In [ ]:
def generate_submission(model,df_test,x_test_features,filename):

  predictions = model.predict(x_test_features)
  df_test['credit_card_default'] = predictions

  results = pd.DataFrame()
  results['customer_id'] = df_test['customer_id']
  results['credit_card_default'] = df_test['credit_card_default']
  results.head(5)

  results.to_csv(filename,index=False)
  print('File Saved ---->>> ',filename)



# Prepare Data for Model Training

**Load Data**

In [ ]:
df_train = df_train_backup
df_test = df_test_backup

In [ ]:
df_test = df_test_backup

In [ ]:
df_test.shape

(11383, 19)

**Under-Sampling Data**

In [ ]:
#separate the dataset
default_df = df_train.loc[df_train['credit_card_default'] == 1]
non_default_df = df_train.loc[df_train['credit_card_default'] == 0]

In [ ]:
normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=42)

new_df.head()

**Transform Data**

In [ ]:
#Replace Missing Values
df_train = replace_NA(df_train)
df_test = replace_NA(df_test)

In [ ]:
#Encode Categorical Columns
df_train, df_test = encode_columns(df_train, df_test)

In [ ]:
#Features & Labels
x_train_features, y_train_labels = create_features_and_labels(df_train,'X')

x_test_features = create_features_and_labels(df_test,'')

(45528, 15) (45528,)
(11383, 15)


**Generate Testing & Training Dataset**

In [ ]:
#split the data
x_train, x_test, y_train, y_test = train_test_split(x_train_features, y_train_labels, test_size=0.20, random_state=0)

# TEST #1 : Baseline Model (LR)

**Model Training & Predictions**

In [ ]:
model_lr_1 = baseline_model(x_train,y_train)
generate_predictions(model_lr_1,x_test,y_test)

/n Accuracy =  95.69514605754448
/nClassification Report
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      8354
           1       0.79      0.65      0.71       752

    accuracy                           0.96      9106
   macro avg       0.88      0.82      0.85      9106
weighted avg       0.95      0.96      0.96      9106



**Model Training on the entire Training Data**

In [ ]:
model_lr = baseline_model(x_train_features,y_train_labels)

**Generate submission file**

In [ ]:
generate_submission(model_lr,df_test,x_test_features,'test1_lr.csv')

File Saved ---->>>  test1_lr.csv


**Score : 82.41476** 

# TEST #2 : Random Forest

**Model Training & Predictions**

In [ ]:
model_rf_1 = random_forest(x_train,y_train,200)
generate_predictions(model_rf_1,x_test,y_test)

Accuracy =  97.90248188007908
Classification Report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8363
           1       0.99      0.75      0.85       743

    accuracy                           0.98      9106
   macro avg       0.99      0.87      0.92      9106
weighted avg       0.98      0.98      0.98      9106



**Model Training on the entire Training Data**

In [ ]:
model_rf = random_forest(x_train_features,y_train_labels,200)

**Generate submission file**

In [ ]:
generate_submission(model_rf,df_test,x_test_features,'test2_rf.csv')

File Saved ---->>>  test2_rf.csv


**Score : 91.74152**

# TEST #3 : Random Forest ( Tree Count = 2000 )

**Model Training & Predictions**

In [ ]:
model_rf_2 = random_forest(x_train,y_train,2000)
generate_predictions(model_rf_2,x_test,y_test)

Accuracy =  98.04524489347682
Classification Report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8388
           1       0.99      0.76      0.86       718

    accuracy                           0.98      9106
   macro avg       0.98      0.88      0.92      9106
weighted avg       0.98      0.98      0.98      9106



**Model Training on the entire Training Data**

In [ ]:
model_rf = random_forest(x_train_features,y_train_labels,2000)

**Generate submission file**

In [ ]:
generate_submission(model_rf,df_test,x_test_features,'test2_rf_HT.csv')

File Saved ---->>>  test2_rf_HT.csv


**Score : 92.08207**

# Oversampling - SMOTE

In [ ]:
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

In [ ]:
#Replace Missing Values
df_train = replace_NA(df_train)
df_test = replace_NA(df_test)

In [ ]:
#Encode Categorical Columns
df_train, df_test = encode_columns(df_train, df_test)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

df = df_train

print('No Defaults', round(df[LABEL_COL].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Defaults', round(df[LABEL_COL].value_counts()[1]/len(df) * 100,2), '% of the dataset')

X = df.drop({LABEL_COL,'customer_id','name', 'occupation_type'}, axis=1)
y = df[LABEL_COL]

sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the Distribution of the labels


# Turn into an array
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain))
print(test_counts_label/ len(original_ytest))

No Defaults 91.88 % of the dataset
Defaults 8.12 % of the dataset
Train: [ 9106  9107  9108 ... 45525 45526 45527] Test: [   0    1    2 ... 9104 9105 9117]
Train: [    0     1     2 ... 45525 45526 45527] Test: [ 9106  9107  9108 ... 18481 18532 18544]
Train: [    0     1     2 ... 45525 45526 45527] Test: [18192 18193 18194 ... 27673 27677 27678]
Train: [    0     1     2 ... 45525 45526 45527] Test: [27291 27292 27293 ... 36840 36843 36846]
Train: [    0     1     2 ... 36840 36843 36846] Test: [36381 36382 36383 ... 45525 45526 45527]
----------------------------------------------------------------------------------------------------
Label Distributions: 

[0.91878758 0.08121242]
[0.9188358 0.0811642]


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV


print('Length of X (train): {} | Length of y (train): {}'.format(len(original_Xtrain), len(original_ytrain)))
print('Length of X (test): {} | Length of y (test): {}'.format(len(original_Xtest), len(original_ytest)))


# List to append the score and then find the average
accuracy_lst = []
precision_lst = []
recall_lst = []
f1_lst = []
auc_lst = []

# Classifier with optimal parameters
# log_reg_sm = grid_log_reg.best_estimator_
#log_reg_sm = LogisticRegression()
#log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
#log_reg_params = {"n_estimators": [1000, 2000,5000,10000]}
log_reg_params = {"n_estimators": [5000]}

rf_reg_sm = RandomForestClassifier(n_estimators = 2000)
# Implementing SMOTE Technique 
# Cross Validating the right way
# Parameters

#rand_log_reg = RandomizedSearchCV(LogisticRegression(), log_reg_params, n_iter=4)
rand_log_reg = RandomizedSearchCV(RandomForestClassifier(),log_reg_params, n_iter=4)

for train, test in sss.split(original_Xtrain, original_ytrain):
    pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), rand_log_reg) # SMOTE happens during Cross Validation not before..
    model = pipeline.fit(original_Xtrain[train], original_ytrain[train])
    best_est = rand_log_reg.best_estimator_
    prediction = best_est.predict(original_Xtrain[test])
  
    accuracy_lst.append(pipeline.score(original_Xtrain[test], original_ytrain[test]))
    precision_lst.append(precision_score(original_ytrain[test], prediction))
    recall_lst.append(recall_score(original_ytrain[test], prediction))
    f1_lst.append(f1_score(original_ytrain[test], prediction))
    auc_lst.append(roc_auc_score(original_ytrain[test], prediction))
  
print('---' * 45)
print('')
print("accuracy: {}".format(np.mean(accuracy_lst)))
print("precision: {}".format(np.mean(precision_lst)))
print("recall: {}".format(np.mean(recall_lst)))
print("f1: {}".format(np.mean(f1_lst)))
print('---' * 45)

Length of X (train): 36423 | Length of y (train): 36423
Length of X (test): 9105 | Length of y (test): 9105


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 1 is smaller than n_iter=4. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 1 is smaller than n_iter=4. Running 1 iterations. For exhaustiv

---------------------------------------------------------------------------------------------------------------------------------------

accuracy: 0.9796557473870203
precision: 0.960313286690822
recall: 0.7819465404490785
f1: 0.8619096278185729
---------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
# Logistic Regression
t0 = time.time()
log_reg_sm = grid_log_reg.best_estimator_
log_reg_sm.fit(Xsm_train, ysm_train)
t1 = time.time()
print("Fitting oversample data took :{} sec".format(t1 - t0))

In [ ]:
labels = [0,1]
smote_prediction = best_est.predict(original_Xtest)
print(classification_report(original_ytest, smote_prediction))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      8366
           1       0.81      0.84      0.82       739

   micro avg       0.97      0.97      0.97      9105
   macro avg       0.90      0.91      0.90      9105
weighted avg       0.97      0.97      0.97      9105



In [ ]:
print(generate_predictions(best_est,original_Xtest,original_ytest))

Accuracy =  98.08896210873147
Classification Report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8366
           1       0.97      0.79      0.87       739

   micro avg       0.98      0.98      0.98      9105
   macro avg       0.98      0.89      0.93      9105
weighted avg       0.98      0.98      0.98      9105

None


In [ ]:
y_score = best_est.decision_function(original_Xtest)
average_precision = average_precision_score(original_ytest, y_score)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

**Prediction**

In [ ]:
x_test_features = create_features_and_labels(df_test,'')

(11383, 15)


**Generate Submission**

In [ ]:
generate_submission(best_est,df_test,x_test_features,'OUTPUT_SMOTE_RF.csv')

File Saved ---->>>  OUTPUT_SMOTE_RF.csv


In [ ]:
from google.colab import files
files.download('OUTPUT_SMOTE_RF.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Handling Unbalanced Dataset ( k-Means SMOTE)

**SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote_ = SMOTE(random_state = 42)
x_train_v2, y_train_v2 = smote_.fit_resample(x_train,y_train)
print(y_train_v2.value_counts)
x_train_v3, y_train_v3 = smote_.fit_resample(x_train_features,y_train_labels)
print(y_train_v3.value_counts)

<bound method IndexOpsMixin.value_counts of 0        0
1        0
2        0
3        0
4        0
        ..
66951    1
66952    1
66953    1
66954    1
66955    1
Name: credit_card_default, Length: 66956, dtype: int64>
<bound method IndexOpsMixin.value_counts of 0        0
1        0
2        0
3        0
4        0
        ..
83657    1
83658    1
83659    1
83660    1
83661    1
Name: credit_card_default, Length: 83662, dtype: int64>


In [ ]:
model_rf_v1 = random_forest(x_train_v2,y_train_v2,2000)
generate_predictions(model_rf_v1,x_test,y_test)

Accuracy =  97.2765209751812
Classification Report
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      8353
           1       0.84      0.84      0.84       753

    accuracy                           0.97      9106
   macro avg       0.91      0.91      0.91      9106
weighted avg       0.97      0.97      0.97      9106



In [ ]:
model_rf_v2 = random_forest(x_train_v3,y_train_v3,2000)

In [ ]:
generate_submission(model_rf_v2,df_test,x_test_features,'test_smote_1.csv')

File Saved ---->>>  test_smote_1.csv


**k-Means SMOTE**

In [ ]:
from kmeans_smote import KMeansSMOTE

In [ ]:
kmeans_smote = KMeansSMOTE(
    kmeans_args={
        'n_clusters': 100
    },
    smote_args={
        'k_neighbors': 10
    }
)
x_train_v2, y_train_v2 = kmeans_smote.fit_resample(x_train,y_train)
x_train_v3, y_train_v3 = kmeans_smote.fit_resample(x_train_features,y_train_labels)

/usr/local/lib/python3.7/dist-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 1. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))
/usr/local/lib/python3.7/dist-packages/kmeans_smote.py:322: UserWarning: No minority clusters found for class 1. Performing regular SMOTE. Try changing the number of clusters.
  warnings.warn('No minority clusters found for class {}. Performing regular SMOTE. Try changing the number of clusters.'.format(minority_class_label))


In [ ]:
model_rf_v1 = random_forest(x_train_v2,y_train_v2,2000)
generate_predictions(model_rf_v1,x_test,y_test)

Accuracy =  97.86953656929497
Classification Report
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8345
           1       0.95      0.78      0.86       761

   micro avg       0.98      0.98      0.98      9106
   macro avg       0.97      0.89      0.92      9106
weighted avg       0.98      0.98      0.98      9106



In [ ]:
model_rf_v2 = random_forest(x_train_v3,y_train_v3,2000)

In [ ]:
generate_submission(model_rf_v2,df_test,x_test_features,'test_smote_knn.csv')

File Saved ---->>>  test_smote_knn.csv


In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(n_estimators=4000, learning_rate=0.05)

In [ ]:
model = xgb.fit(x_train,y_train)

In [ ]:
generate_predictions(model,x_test,y_test)

Accuracy =  97.68284647485174
Classification Report
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      8345
           1       0.91      0.80      0.85       761

   micro avg       0.98      0.98      0.98      9106
   macro avg       0.95      0.90      0.92      9106
weighted avg       0.98      0.98      0.98      9106



In [ ]:
model_xgb = xgb.fit(x_train_features,y_train_labels)

In [ ]:
generate_submission(model_xgb,df_test,x_test_features,'test_smote_xgb2.csv')

File Saved ---->>>  test_smote_xgb2.csv


# TEST #4 : All 4 Classifiers

In [ ]:
# Let's implement simple classifiers

classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "KNearest": KNeighborsClassifier(),
    "Support Vector Classifier": SVC(),
    "DecisionTreeClassifier": DecisionTreeClassifier()
}

In [ ]:
# Wow our scores are getting even high scores even when applying cross validation.
from sklearn.model_selection import cross_val_score


for key, classifier in classifiers.items():
    classifier.fit(X_train, y_train)
    training_score = cross_val_score(classifier, X_train, y_train, cv=5)
    print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")